# ベンチマークモデル

基本となるモデル作成

# ライブラリとデータの読み込み

In [110]:
import numpy as np
import pandas as pd

In [111]:
data_dir = "../data/input/"


train_raw = pd.read_csv(data_dir+"train.csv")
test_raw = pd.read_csv(data_dir+"test.csv")

df = pd.concat([train_raw.drop(["id","price"],axis=1), test_raw.drop("id",axis=1)], axis=0)

# データ整形・特徴量エンジニアリング

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53940 entries, 0 to 16181
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   x        53940 non-null  float64
 7   y        53940 non-null  float64
 8   z        53940 non-null  float64
dtypes: float64(6), object(3)
memory usage: 4.1+ MB


欠損値はとりあえずないのでそのまま学習してみる。
あとはカテゴリカル変数をうまく扱えば良い。

In [113]:
# ワンホットエンコーディングでカテゴリカル変数をうまく処理
df = pd.get_dummies(df)

In [114]:
df.head()

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.00,62.9,59.0,6.39,6.30,3.99,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,1.16,61.9,59.0,6.71,6.66,4.14,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,0.74,62.3,60.0,5.78,5.80,3.61,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,2.17,62.2,59.0,8.28,8.34,5.17,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,0.31,57.6,62.0,4.46,4.50,2.58,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [115]:
train = df[:train_raw.shape[0]]
test = df[train_raw.shape[0]:]

# モデルの作成と学習 

In [116]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [117]:
def train_and_predict(X_train, X_valid, y_train, y_valid, X_test, params):

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)


    # 上記のパラメータでモデルを学習する
    model = lgb.train(
        params, lgb_train,
        # モデルの評価用データを渡す
        valid_sets=lgb_eval,
        # 最大で 5000 ラウンドまで学習する
        num_boost_round=5000,
        # 50 ラウンド経過しても性能が向上しないときは学習を打ち切る
        early_stopping_rounds=50,
    )

    # valid を予測する
    y_valid_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    # テストデータを予測する
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    return y_pred, y_valid_pred, model

In [125]:
params = {
        "learning_rate": 0.1,
        "num_leaves": 8,
        "boosting_type": "gbdt",
        "colsample_bytree": 0.65,
        "reg_alpha": 1,
        "reg_lambda": 1,
        "objective": "root_mean_squared_error",
        "random_state": 42
}

In [128]:
# 学習前にlog(y+1)で変換（このまま予測して RMSE で評価すると RMSLE と同じになる。提出前の最後に exp で戻す）
target = np.log(train_raw["price"] + 1)  # np.log1p() でもOK


y_preds = []
oof = np.zeros(len(target)) # oof: out of folds のこと
models = []
kf = KFold(n_splits=5, shuffle=True, random_state=0)
for train_index, valid_index in kf.split(target):
    X_train, X_valid = (train.iloc[train_index, :], train.iloc[valid_index, :])
    y_train, y_valid = (target.iloc[train_index], target.iloc[valid_index])

    # lgbmの実行
    y_pred, y_valid_pred, model = train_and_predict(X_train, X_valid, y_train, y_valid, test, params)

    # 結果の保存
    y_preds.append(y_pred)
    oof[valid_index] = y_valid_pred
    models.append(model)



[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000501 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1268
[LightGBM] [Info] Number of data points in the train set: 30206, number of used features: 26
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 7.779973
[1]	valid_0's rmse: 0.919406
Training until validation scores don't improve for 50 rounds
[2]	valid_0's rmse: 0.835529
[3]	valid_0's rmse: 0.760359
[4]	valid_0's rmse: 0.693594
[5]	valid_0's rmse: 0.634059
[6]	valid_0's rmse: 0.581459
[7]	valid_0's rmse: 0.533996
[8]	valid_0's rmse: 0.492616
[9]	valid_0's rmse: 0.455813
[10]	valid_0's rmse: 0.423228
[11]	valid_0's rmse: 0.394653
[12]	valid_0's rmse: 0.369631
[13]	valid_0's rmse

[256]	valid_0's rmse: 0.0990607
[257]	valid_0's rmse: 0.0988949
[258]	valid_0's rmse: 0.0988364
[259]	valid_0's rmse: 0.0987833
[260]	valid_0's rmse: 0.0987466
[261]	valid_0's rmse: 0.0987088
[262]	valid_0's rmse: 0.0986751
[263]	valid_0's rmse: 0.0986001
[264]	valid_0's rmse: 0.0985804
[265]	valid_0's rmse: 0.0985038
[266]	valid_0's rmse: 0.0984632
[267]	valid_0's rmse: 0.0983296
[268]	valid_0's rmse: 0.0983137
[269]	valid_0's rmse: 0.0983063
[270]	valid_0's rmse: 0.0982832
[271]	valid_0's rmse: 0.0982183
[272]	valid_0's rmse: 0.0981555
[273]	valid_0's rmse: 0.0981252
[274]	valid_0's rmse: 0.0981026
[275]	valid_0's rmse: 0.0980781
[276]	valid_0's rmse: 0.0980616
[277]	valid_0's rmse: 0.0980349
[278]	valid_0's rmse: 0.0980212
[279]	valid_0's rmse: 0.0979623
[280]	valid_0's rmse: 0.0979459
[281]	valid_0's rmse: 0.097918
[282]	valid_0's rmse: 0.0978731
[283]	valid_0's rmse: 0.0978513
[284]	valid_0's rmse: 0.0978367
[285]	valid_0's rmse: 0.0977983
[286]	valid_0's rmse: 0.0977697
[287]	val

[605]	valid_0's rmse: 0.091956
[606]	valid_0's rmse: 0.0919481
[607]	valid_0's rmse: 0.0919367
[608]	valid_0's rmse: 0.0919333
[609]	valid_0's rmse: 0.091914
[610]	valid_0's rmse: 0.0919071
[611]	valid_0's rmse: 0.0918956
[612]	valid_0's rmse: 0.0918656
[613]	valid_0's rmse: 0.0918585
[614]	valid_0's rmse: 0.091843
[615]	valid_0's rmse: 0.0918422
[616]	valid_0's rmse: 0.09181
[617]	valid_0's rmse: 0.0917976
[618]	valid_0's rmse: 0.0917945
[619]	valid_0's rmse: 0.0917905
[620]	valid_0's rmse: 0.0917902
[621]	valid_0's rmse: 0.091785
[622]	valid_0's rmse: 0.0917816
[623]	valid_0's rmse: 0.0917685
[624]	valid_0's rmse: 0.0917416
[625]	valid_0's rmse: 0.0917243
[626]	valid_0's rmse: 0.0916971
[627]	valid_0's rmse: 0.0916966
[628]	valid_0's rmse: 0.0916975
[629]	valid_0's rmse: 0.0916945
[630]	valid_0's rmse: 0.0916919
[631]	valid_0's rmse: 0.0916806
[632]	valid_0's rmse: 0.0916746
[633]	valid_0's rmse: 0.0916555
[634]	valid_0's rmse: 0.0916417
[635]	valid_0's rmse: 0.0916399
[636]	valid_0'

[921]	valid_0's rmse: 0.0904999
[922]	valid_0's rmse: 0.0904974
[923]	valid_0's rmse: 0.0904963
[924]	valid_0's rmse: 0.0904887
[925]	valid_0's rmse: 0.0904792
[926]	valid_0's rmse: 0.0904765
[927]	valid_0's rmse: 0.0904779
[928]	valid_0's rmse: 0.0904695
[929]	valid_0's rmse: 0.0904681
[930]	valid_0's rmse: 0.0904608
[931]	valid_0's rmse: 0.0904617
[932]	valid_0's rmse: 0.0904581
[933]	valid_0's rmse: 0.0904508
[934]	valid_0's rmse: 0.0904507
[935]	valid_0's rmse: 0.0904524
[936]	valid_0's rmse: 0.0904514
[937]	valid_0's rmse: 0.0904541
[938]	valid_0's rmse: 0.0904492
[939]	valid_0's rmse: 0.0904454
[940]	valid_0's rmse: 0.0904441
[941]	valid_0's rmse: 0.0904388
[942]	valid_0's rmse: 0.0904394
[943]	valid_0's rmse: 0.0904384
[944]	valid_0's rmse: 0.0904414
[945]	valid_0's rmse: 0.0904382
[946]	valid_0's rmse: 0.0904332
[947]	valid_0's rmse: 0.0904332
[948]	valid_0's rmse: 0.0904254
[949]	valid_0's rmse: 0.0904232
[950]	valid_0's rmse: 0.0904246
[951]	valid_0's rmse: 0.0904197
[952]	va

[1221]	valid_0's rmse: 0.0898255
[1222]	valid_0's rmse: 0.0898217
[1223]	valid_0's rmse: 0.0898201
[1224]	valid_0's rmse: 0.0898182
[1225]	valid_0's rmse: 0.0898186
[1226]	valid_0's rmse: 0.0898185
[1227]	valid_0's rmse: 0.0898202
[1228]	valid_0's rmse: 0.0898189
[1229]	valid_0's rmse: 0.0898196
[1230]	valid_0's rmse: 0.0898161
[1231]	valid_0's rmse: 0.0898129
[1232]	valid_0's rmse: 0.089809
[1233]	valid_0's rmse: 0.0898087
[1234]	valid_0's rmse: 0.0898085
[1235]	valid_0's rmse: 0.089808
[1236]	valid_0's rmse: 0.089806
[1237]	valid_0's rmse: 0.0898054
[1238]	valid_0's rmse: 0.0897975
[1239]	valid_0's rmse: 0.08979
[1240]	valid_0's rmse: 0.0897956
[1241]	valid_0's rmse: 0.0898
[1242]	valid_0's rmse: 0.0898004
[1243]	valid_0's rmse: 0.0898002
[1244]	valid_0's rmse: 0.0897991
[1245]	valid_0's rmse: 0.0897988
[1246]	valid_0's rmse: 0.0897984
[1247]	valid_0's rmse: 0.0897976
[1248]	valid_0's rmse: 0.0897988
[1249]	valid_0's rmse: 0.0897972
[1250]	valid_0's rmse: 0.0898003
[1251]	valid_0's r

...

[1136]	valid_0's rmse: 0.0880526
[1137]	valid_0's rmse: 0.0880485
[1138]	valid_0's rmse: 0.088048
[1139]	valid_0's rmse: 0.088048
[1140]	valid_0's rmse: 0.0880484
[1141]	valid_0's rmse: 0.0880461
[1142]	valid_0's rmse: 0.088044
[1143]	valid_0's rmse: 0.0880422
[1144]	valid_0's rmse: 0.0880401
[1145]	valid_0's rmse: 0.0880327
[1146]	valid_0's rmse: 0.0880317
[1147]	valid_0's rmse: 0.0880315
[1148]	valid_0's rmse: 0.0880311
[1149]	valid_0's rmse: 0.0880314
[1150]	valid_0's rmse: 0.0880313
[1151]	valid_0's rmse: 0.0880301
[1152]	valid_0's rmse: 0.0880228
[1153]	valid_0's rmse: 0.0880235
[1154]	valid_0's rmse: 0.0880205
[1155]	valid_0's rmse: 0.0880204
[1156]	valid_0's rmse: 0.0880178
[1157]	valid_0's rmse: 0.08802
[1158]	valid_0's rmse: 0.0880167
[1159]	valid_0's rmse: 0.0880161
[1160]	valid_0's rmse: 0.0880117
[1161]	valid_0's rmse: 0.088012
[1162]	valid_0's rmse: 0.0880115
[1163]	valid_0's rmse: 0.0880104
[1164]	valid_0's rmse: 0.0880089
[1165]	valid_0's rmse: 0.0880075
[1166]	valid_0's

[1421]	valid_0's rmse: 0.0876623
[1422]	valid_0's rmse: 0.0876632
[1423]	valid_0's rmse: 0.0876624
[1424]	valid_0's rmse: 0.0876623
[1425]	valid_0's rmse: 0.0876618
[1426]	valid_0's rmse: 0.0876626
[1427]	valid_0's rmse: 0.0876628
[1428]	valid_0's rmse: 0.087663
[1429]	valid_0's rmse: 0.0876632
[1430]	valid_0's rmse: 0.0876639
[1431]	valid_0's rmse: 0.0876629
[1432]	valid_0's rmse: 0.0876611
[1433]	valid_0's rmse: 0.0876627
[1434]	valid_0's rmse: 0.0876602
[1435]	valid_0's rmse: 0.0876588
[1436]	valid_0's rmse: 0.0876572
[1437]	valid_0's rmse: 0.087655
[1438]	valid_0's rmse: 0.0876558
[1439]	valid_0's rmse: 0.087656
[1440]	valid_0's rmse: 0.0876527
[1441]	valid_0's rmse: 0.087652
[1442]	valid_0's rmse: 0.0876521
[1443]	valid_0's rmse: 0.0876483
[1444]	valid_0's rmse: 0.0876472
[1445]	valid_0's rmse: 0.0876495
[1446]	valid_0's rmse: 0.0876465
[1447]	valid_0's rmse: 0.0876472
[1448]	valid_0's rmse: 0.0876444
[1449]	valid_0's rmse: 0.0876416
[1450]	valid_0's rmse: 0.0876419
[1451]	valid_0

[1746]	valid_0's rmse: 0.0874124
[1747]	valid_0's rmse: 0.0874082
[1748]	valid_0's rmse: 0.0874084
[1749]	valid_0's rmse: 0.0874047
[1750]	valid_0's rmse: 0.087404
[1751]	valid_0's rmse: 0.0874043
[1752]	valid_0's rmse: 0.0874032
[1753]	valid_0's rmse: 0.0874012
[1754]	valid_0's rmse: 0.0874011
[1755]	valid_0's rmse: 0.0873992
[1756]	valid_0's rmse: 0.0873957
[1757]	valid_0's rmse: 0.087394
[1758]	valid_0's rmse: 0.0873947
[1759]	valid_0's rmse: 0.0873936
[1760]	valid_0's rmse: 0.0873949
[1761]	valid_0's rmse: 0.0873941
[1762]	valid_0's rmse: 0.0873947
[1763]	valid_0's rmse: 0.0873932
[1764]	valid_0's rmse: 0.0873899
[1765]	valid_0's rmse: 0.087388
[1766]	valid_0's rmse: 0.0873892
[1767]	valid_0's rmse: 0.0873895
[1768]	valid_0's rmse: 0.0873892
[1769]	valid_0's rmse: 0.0873891
[1770]	valid_0's rmse: 0.0873898
[1771]	valid_0's rmse: 0.08739
[1772]	valid_0's rmse: 0.0873868
[1773]	valid_0's rmse: 0.0873912
[1774]	valid_0's rmse: 0.0873932
[1775]	valid_0's rmse: 0.0873936
[1776]	valid_0'

[2006]	valid_0's rmse: 0.0872307
[2007]	valid_0's rmse: 0.0872317
[2008]	valid_0's rmse: 0.0872304
[2009]	valid_0's rmse: 0.0872303
[2010]	valid_0's rmse: 0.0872301
[2011]	valid_0's rmse: 0.0872296
[2012]	valid_0's rmse: 0.0872262
[2013]	valid_0's rmse: 0.0872276
[2014]	valid_0's rmse: 0.087226
[2015]	valid_0's rmse: 0.0872231
[2016]	valid_0's rmse: 0.0872219
[2017]	valid_0's rmse: 0.0872216
[2018]	valid_0's rmse: 0.0872212
[2019]	valid_0's rmse: 0.0872208
[2020]	valid_0's rmse: 0.0872197
[2021]	valid_0's rmse: 0.0872215
[2022]	valid_0's rmse: 0.0872214
[2023]	valid_0's rmse: 0.0872229
[2024]	valid_0's rmse: 0.0872226
[2025]	valid_0's rmse: 0.0872244
[2026]	valid_0's rmse: 0.0872243
[2027]	valid_0's rmse: 0.0872247
[2028]	valid_0's rmse: 0.0872231
[2029]	valid_0's rmse: 0.0872241
[2030]	valid_0's rmse: 0.0872232
[2031]	valid_0's rmse: 0.0872236
[2032]	valid_0's rmse: 0.0872236
[2033]	valid_0's rmse: 0.087223
[2034]	valid_0's rmse: 0.0872235
[2035]	valid_0's rmse: 0.0872237
[2036]	valid

[2256]	valid_0's rmse: 0.0871648
[2257]	valid_0's rmse: 0.0871641
[2258]	valid_0's rmse: 0.0871644
[2259]	valid_0's rmse: 0.087166
[2260]	valid_0's rmse: 0.087165
[2261]	valid_0's rmse: 0.0871654
[2262]	valid_0's rmse: 0.0871652
[2263]	valid_0's rmse: 0.0871651
[2264]	valid_0's rmse: 0.0871668
[2265]	valid_0's rmse: 0.0871684
[2266]	valid_0's rmse: 0.0871681
[2267]	valid_0's rmse: 0.0871684
[2268]	valid_0's rmse: 0.0871679
[2269]	valid_0's rmse: 0.0871683
[2270]	valid_0's rmse: 0.087168
[2271]	valid_0's rmse: 0.0871661
[2272]	valid_0's rmse: 0.087164
[2273]	valid_0's rmse: 0.0871638
[2274]	valid_0's rmse: 0.0871614
[2275]	valid_0's rmse: 0.0871635
[2276]	valid_0's rmse: 0.0871635
[2277]	valid_0's rmse: 0.0871634
[2278]	valid_0's rmse: 0.0871636
[2279]	valid_0's rmse: 0.0871633
[2280]	valid_0's rmse: 0.0871642
[2281]	valid_0's rmse: 0.0871653
[2282]	valid_0's rmse: 0.0871654
[2283]	valid_0's rmse: 0.0871661
[2284]	valid_0's rmse: 0.0871666
[2285]	valid_0's rmse: 0.0871677
[2286]	valid_0

In [129]:
# CVスコア

scores = [
    m.best_score['valid_0']['rmse'] for m in models
]

score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print("mean: ", score)
print("direct: ", rmse(target, oof))

===CV scores===
[0.08934606822357208, 0.08825204296097845, 0.09126245955178447, 0.0903570797837919, 0.08716058550630076]
mean:  0.08927564720528554
direct:  0.08928759580689283


In [130]:
# submission の作成

sub = pd.DataFrame(test_raw["id"])
y_sub = sum(y_preds) / len(y_preds)

# 最後に、予測結果に対しexp(y)-1で逆変換
y_sub = np.exp(y_sub) - 1  # np.expm1() でもOK

sub["price"] = y_sub
sub

,id,price
0,37758,518.737004
1,37759,2003.037306
2,37760,11637.025041
3,37761,4076.821336
4,37762,6013.030004
...,...,...
16177,53935,8591.479905
16178,53936,6662.198400
16179,53937,2424.172128
16180,53938,729.485800


In [131]:
sub.to_csv(
    'submission.csv',
    index=False
)